# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto el triunfo,13.2748,-88.5472,32.59,43,54,2.90,SV,1708035967
1,1,olonkinbyen,70.9224,-8.7160,-11.07,57,100,11.51,SJ,1708035968
2,2,port-aux-francais,-49.3498,70.2200,8.82,94,97,14.97,TF,1708035968
3,3,tahe,52.3696,124.5765,-19.77,98,100,0.81,CN,1708035969
4,4,beau vallon,-20.4298,57.7118,27.23,89,75,2.06,MU,1708035969


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 28) & (city_data_df['Wind Speed'] < 4.5)
                            & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_df = ideal_df.dropna(how='any')

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
117,117,dalcahue,-42.3796,-73.6473,22.06,72,0,2.57,CL,1708036019
119,119,lake havasu city,34.4780,-114.3176,22.15,27,0,1.34,US,1708035769
265,265,villa ojo de agua,-29.5027,-63.6951,26.23,43,0,4.32,AR,1708036077
318,318,medina,24.4712,39.6111,21.17,23,0,3.09,SA,1708036105
397,397,papao,12.6919,-16.3792,27.06,78,0,2.06,SN,1708036193
512,512,san luis,-33.3020,-66.3370,26.84,40,0,2.00,AR,1708036273


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
117,dalcahue,CL,-42.3796,-73.6473,72,
119,lake havasu city,US,34.4780,-114.3176,27,
265,villa ojo de agua,AR,-29.5027,-63.6951,43,
318,medina,SA,24.4712,39.6111,23,
397,papao,SN,12.6919,-16.3792,78,
512,san luis,AR,-33.3020,-66.3370,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
dalcahue - nearest hotel: No hotel found
lake havasu city - nearest hotel: No hotel found
villa ojo de agua - nearest hotel: No hotel found
medina - nearest hotel: No hotel found
papao - nearest hotel: No hotel found
san luis - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
117,dalcahue,CL,-42.3796,-73.6473,72,No hotel found
119,lake havasu city,US,34.4780,-114.3176,27,No hotel found
265,villa ojo de agua,AR,-29.5027,-63.6951,43,No hotel found
318,medina,SA,24.4712,39.6111,23,No hotel found
397,papao,SN,12.6919,-16.3792,78,No hotel found
512,san luis,AR,-33.3020,-66.3370,40,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display
# Drop cities without a hotel found
hotel_df_clean = hotel_df.loc[hotel_df['Hotel Name'] != 'No hotel found']

# Configure the map plot
map_plot2 = hotel_df_clean.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)